# Using Scaler Arguments in MESSAGEix

This notebook outlines the steps required to run a MESSAGEix model and scenario using scaler arguments, which help improve the condition of the coefficient matrix for solving in GAMS. These steps involve the use of the `LPdiag` and `make_scaler` tools.

- `LPdiag` reads the MPS file and identifies rows and columns with outlier values.
- `make_scaler` uses the information from LPdiag to generate a set of scaling arguments, stored in a `.gms` file. These arguments are then used by GAMS to scale the coefficients of rows and columns during model solution.

## Steps

### 1. **Create the MPS file**

Generate an MPS file for a base scenario. This file will be used to derive the scaling arguments.

> ⚠️ You do **not** need to generate scaling arguments for every scenario you want to run. Creating them can take around **2 hours**, so it's more efficient to generate them once for a **base scenario** (e.g., `model: SSP2 | scenario: baseline`) and reuse them for derivative scenarios (e.g., `1000f`, `600f`, etc.).

### 2. **Create scaler arguments**

Use the `make_scaler` tool with the MPS file from Step 1 to generate the scaling arguments. These will be stored in the `.../message_ix/model/scaler/` directory.

### 3. **Run the scenarios**

Once the scaler arguments have been created, you can use them when solving the base and derivative scenarios.

- Use the function `get_scaler_args(scenario_parent)`, where `scenario_parent` refers to the MESSAGEix scenario object for which the scaler arguments were generated (e.g., `model: SSP2 | scenario: baseline`).
- Pass the result of `get_scaler_args(scenario_parent)` to the `gams_args` parameter in the solve statement.

> ⚠️ It is important to use the standard GAMS/CPLEX settings in the solve statement:
> 
> ```python
> gams_args={"lpmethod": "0", "scaind": "0"}
> ```
> 
> These settings allow GAMS to automatically choose the best LP method and scaling strategy. When running in multiple threads, this enables GAMS/CPLEX to try different methods, avoiding poor performance from a fixed strategy.


In [1]:
import pandas as pd
import ixmp
import message_ix
import os

from message_ix.tools.lp_diag import LPdiag
from message_ix.tools.make_scaler import make_scaler
from message_ix.tools.make_scaler import show_range
from message_ix.tools.make_scaler import get_scaler_args

from message_ix.util import make_df

%matplotlib inline

lp = LPdiag()
mp = ixmp.Platform()

## Generating model-scenario mps file

In [ ]:
ssp = "SSP2"
base = message_ix.Scenario(mp, model=f"SSP_{ssp}_development", scenario="baseline_DEFAULT_step_15")

# creating unscaled scenario
model_name = f"ScalingSSP_{ssp}_development"
scenario_name = f"baseline_DEFAULT_step_15_ywp"
scen = base.clone(
    model_name,
    scenario_name,
    f"unscaled {ssp} model",
    keep_solution = False,)

scen.check_out()

scen.commit(comment= f"unscaled {ssp} scenario")
scen.set_as_default()

current_directory = os.getcwd()
mps_dir = os.path.join(current_directory, f"{model_name}_{scenario_name}.mps")

scen.solve(solve_options={'barcrossalg': '2',
                          'scaind':'-1',
                          "writemps": mps_dir,
                         })
scen.var("OBJ")["lvl"]

# Create scaler arguments

In [2]:
ssp = "SSP2"
model_name = f"ScalingSSP_{ssp}_development"
scenario_name = f"baseline_DEFAULT_step_15_ywp"
scen = message_ix.Scenario(mp, model=model_name, scenario=scenario_name)


In [3]:
current_directory = os.getcwd()
mps_dir = os.path.join(current_directory, f"{model_name}_{scenario_name}.mps")
scale_df = make_scaler(mps_dir, scen, bounds=4, steps=1)


Reading MPS-format file C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\tools\make_scaler\ScalingSSP_SSP2_development_baseline_DEFAULT_step_15_ywp.mps.
Next section found: NAME          gamsmodel (line 1).
	Problem name: gamsmodel.
Next section found: ROWS (line 2).
	Row _obj (row_seq = 0) is the objective (goal function) row.
Next section found: COLUMNS (line 1819718).
Next section found: RHS (line 8548702).
	Id of RHS: rhs
Next section found: BOUNDS (line 8615990).
	Id of BOUNDS: bnd
Next section found: ENDATA (line 9730612).

Finished processing 9730612 lines of the MPS file: C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\tools\make_scaler\ScalingSSP_SSP2_development_baseline_DEFAULT_step_15_ywp.mps.
LP has: 1819715 rows, 1728169 cols, 6728983 non-zeros, matrix density = 2.14e-06.
Numbers of redefined: RHS = 67287, ranges = 0, bounds = 1114621.

The GF (objective) row named "_obj" has 169 elements.
Distribution of the GF (objective) values:
coun

## Run scenarios with scaled matrix

Results: solution time 131.45s

In [ ]:
sc_name = scenario_name+"-scaled"
sc = scen.clone(
    model_name,
    sc_name,
    "solve scenario with scaler",
    keep_solution=False,
)
sc.check_out()

sc.commit(comment="solve scenario with scaler")
sc.set_as_default()

In [ ]:
argument = get_scaler_args(scen)

sc.solve(gams_args=[argument],
         solve_options={"lpmethod": "0"
                        "scaind": "0",
                       }
        )

In [ ]:
print("scaled objective: ", sc.var("OBJ")["lvl"])
print("unscaled objective: ", scen.var("OBJ")["lvl"])

In [ ]:
mp.close_db()